In [38]:
# prototyoe of indel model
# メモ：議論を簡潔にするためにその他の変異モデルは考えないことにする
import numpy as np
import math

####################
#  some function
####################

def disturibution_of_indel_size(q, r):
    length = np.random.negative_binomial(r,q)
    return length

def insertion_length(time, L, rambda_I, q, r):
    u = disturibution_of_indel_size(q, r)
    prob = [
        1 - math.exp(-1 * rambda_I * time * (L + 1) ), # occur insertion
        math.exp(-1 * rambda_I * time * (L + 1) ) # doesn't occur insertion
    ]
    try:
        tf = np.random.choice([True, False],p=prob)
    except ValueError:
        print(prob)
    if tf:
        return u
    else:
        return 0

def deletion_length(time, L, rambda_D, q, r):
    u = disturibution_of_indel_size(q, r)
    prob = [
        1 - math.exp(-1 * rambda_D * time * ( u - 1 + L ) ), # occur deletion
        math.exp(-1 * rambda_D * time * ( u - 1 + L ) ) # doesn't occur deletion
    ]
    try:
        tf = np.random.choice([True, False],p=prob)
    except ValueError:
        print(prob)
        print(time, rambda_D, u, L)
    if tf:
        return u
    else:
        return 0

def insertion(seq, length):
    if length == 0:
        return seq
    index = len(seq)
    position = np.random.randint(0, index)
    insertseq = "".join(np.random.choice(["A", "G", "C", "T"], length))
    newseq = seq[:position] + insertseq + seq[position:] 
    # print(seq, "=>", newseq)
    return newseq

def deletion(seq, length):
    if length == 0:
        return seq
    L = len(seq)
    start_position = max(np.random.randint(-1 * length + 1, L - 1), 0)
    end_position = min(start_position + length - 1, L - 1)
    newseq = seq[:start_position] + seq[end_position + 1:]
    return newseq

def replication(seq, rambda_I, rambda_D, q, r):
    # indel
    time = np.random.exponential()
    in_length = insertion_length(time, len(seq), rambda_I, q, r)
    del_length = deletion_length(time, len(seq), rambda_D, q, r)
    both_are_occur = insertion_length != 0 and deletion_length != 0

    if both_are_occur:
        if np.random.choice([True, False]):
            newseq = insertion(seq, in_length)
            newseq = deletion(newseq, del_length)
        else:
            newseq = deletion(seq, del_length)
            newseq = insertion(newseq, in_length)     
    else:
        # どっちも起きるように見えるけど実際には片方しか起きない
        newseq = insertion(seq, in_length)
        newseq = deletion(newseq, del_length)
    return newseq

####################
#  simulation
####################
N = 1000
L = 10 # sequence length
rambda_I = 0.1
rambda_D = 0.0
q = 0.5
r = 1

initseq = "".join(np.random.choice(["A", "T", "G", "C"], L))

stack = [initseq]
while(len(stack) < N):
    mother = stack.pop()
    daughter_L = replication(mother, rambda_I, rambda_D, q, r)
    daughter_R = replication(mother, rambda_I, rambda_D, q, r)
    stack.extend([daughter_L, daughter_R])

print("Done!")
for idx, seq in enumerate(stack): 
    print(len(seq),":",idx + 1) 


Done!
10 : 1
10 : 2
10 : 3
10 : 4
11 : 5
11 : 6
10 : 7
10 : 8
10 : 9
12 : 10
11 : 11
12 : 12
11 : 13
12 : 14
13 : 15
14 : 16
14 : 17
14 : 18
16 : 19
21 : 20
16 : 21
18 : 22
18 : 23
19 : 24
21 : 25
19 : 26
19 : 27
21 : 28
20 : 29
20 : 30
20 : 31
24 : 32
23 : 33
23 : 34
26 : 35
29 : 36
28 : 37
28 : 38
28 : 39
31 : 40
31 : 41
31 : 42
31 : 43
34 : 44
32 : 45
33 : 46
35 : 47
34 : 48
34 : 49
34 : 50
34 : 51
34 : 52
36 : 53
37 : 54
38 : 55
41 : 56
42 : 57
42 : 58
52 : 59
43 : 60
45 : 61
46 : 62
50 : 63
51 : 64
54 : 65
58 : 66
60 : 67
63 : 68
62 : 69
62 : 70
66 : 71
61 : 72
61 : 73
61 : 74
62 : 75
63 : 76
64 : 77
67 : 78
67 : 79
67 : 80
67 : 81
69 : 82
75 : 83
69 : 84
70 : 85
70 : 86
71 : 87
73 : 88
75 : 89
77 : 90
81 : 91
77 : 92
79 : 93
81 : 94
85 : 95
85 : 96
86 : 97
86 : 98
87 : 99
88 : 100
89 : 101
92 : 102
92 : 103
90 : 104
92 : 105
95 : 106
95 : 107
96 : 108
95 : 109
96 : 110
95 : 111
97 : 112
95 : 113
95 : 114
98 : 115
96 : 116
98 : 117
97 : 118
99 : 119
102 : 120
101 : 121
103 : 122
1